In [1]:
import sys
import xml.sax
import xml.parsers.expat
import time 
import pickle
import os
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer 
from collections import OrderedDict
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

root_path = '/Users/mudit/anlp/News_bias_detector/'
def split_train_test_df(df,tr,te):
  total_rowsn=df.shape[0]

  train_rowsn=int(total_rowsn*tr/100)
  return df[0:train_rowsn],df[train_rowsn+1:total_rowsn]

def split_train_test_arr(arr,tr,te):
  total_rowsn=arr.shape[0]

  train_rowsn=int(total_rowsn*tr/100)
  return arr[0:train_rowsn,:],arr[train_rowsn+1:total_rowsn,:]

In [18]:
text=[]
labels = []
titles = []
class ParseHandler(xml.sax.handler.ContentHandler):
    def __init__(self):
        self.text=""
    def startElement(self, name, attrs):
        self.CurrentData = name
        if(self.CurrentData=="article"):
            labels.append(attrs['hyperpartisan'])



parser = xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_namespaces, 0) 
Handler = ParseHandler()
parser.setContentHandler(Handler)
parser.parse(root_path+"article_labels.xml")


import re
class ParseHandler1(xml.sax.handler.ContentHandler):
    def __init__(self):
        self.text=""
        self.r1=0
    def startElement(self, name, attrs):
        self.CurrentData = name
        if 'title' in attrs.getNames():
            titles.append(attrs.getValue('title'))
    def endElement(self, name):
        if(name=="article"):
            self.text=clean_text(self.text)
            self.text=re.sub(r'([^\s\w]|_)+', '', self.text)
            self.text=self.text.strip()
            text.append(self.text)
            self.text="" 
        
    def characters(self, data):
        self.text += data

def clean_text(text1):
    text1 = text1.replace(".", ". ")
    text1 = text1.replace(" _", " ")
    text1 = text1.replace("  ", " ")
    text1 = text1.replace(". . . .", "...")
    text1 = text1.replace(". . .", "...")
    return text1

parser = xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_namespaces, 0) 
Handler = ParseHandler1()
parser.setContentHandler(Handler)
parser.parse(root_path+"articles-training.xml")

print(len(titles),len(labels),len(titles))

# import pandas as pd 
# df1 = pd.DataFrame(text)
# df2=pd.DataFrame(labels)
# # print(df1)
# result = pd.concat([df1, df2], axis=1, sort=False) 
# result.columns = ['texts','labels']
# result.to_csv('cleaned_data.csv')

645 645 645


In [19]:
df = pd.read_csv('cleaned_data.csv')
texts = df['texts']
labels_df = df['labels']

train_df,test_df = split_train_test_df(texts,80,20)
train_labels_df,test_labels_df = split_train_test_df(labels_df,80,20)

tfidf_vectorizer=TfidfVectorizer(use_idf=True)

train_tfidf_vectors=tfidf_vectorizer.fit_transform(train_df)
test_tfidf_vectors = tfidf_vectorizer.transform(test_df)

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l1', random_state=None, tol=0.0001,
          verbose=0)

In [21]:
lin_clf = LinearSVC(penalty='l1',dual=False)
lin_clf.fit(train_tfidf_vectors, train_labels_df.to_numpy())
test_tfidf_vectors = tfidf_vectorizer.transform(test_df)
out = lin_clf.predict(test_tfidf_vectors)
print('Linear kernel\n', classification_report(out,test_labels_df.to_numpy()))

Linear kernel
               precision    recall  f1-score   support

       False       0.89      0.84      0.86        98
        True       0.56      0.67      0.61        30

    accuracy                           0.80       128
   macro avg       0.72      0.75      0.73       128
weighted avg       0.81      0.80      0.80       128



In [28]:
rbfclf = SVC(kernel = 'rbf',degree=3)
rbfclf.fit(train_tfidf_vectors, train_labels_df.to_numpy())

out = rbfclf.predict(test_tfidf_vectors)
print('SVM with RBF kernel\n', classification_report(out,test_labels_df.to_numpy()))

/Users/mudit/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM with RBF kernel
               precision    recall  f1-score   support

       False       1.00      0.72      0.84       128
        True       0.00      0.00      0.00         0

    accuracy                           0.72       128
   macro avg       0.50      0.36      0.42       128
weighted avg       1.00      0.72      0.84       128



/Users/mudit/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [35]:
from sklearn.ensemble import RandomForestClassifier
rfc_clf = RandomForestClassifier(n_estimators=100)
rfc_clf.fit(train_tfidf_vectors, train_labels_df.to_numpy())

out = rfc_clf.predict(test_tfidf_vectors)
print('Random Forest Classifier\n', classification_report(out,test_labels_df.to_numpy()))

Random Forest Classifier
               precision    recall  f1-score   support

       False       0.92      0.80      0.86       106
        True       0.42      0.68      0.52        22

    accuracy                           0.78       128
   macro avg       0.67      0.74      0.69       128
weighted avg       0.84      0.78      0.80       128



In [36]:
ada_clf = AdaBoostClassifier(n_estimators=100)
ada_clf.fit(train_tfidf_vectors, train_labels_df.to_numpy())

out = ada_clf.predict(test_tfidf_vectors)
print('Adaboost\n', classification_report(out,test_labels_df.to_numpy()))

Adaboost
               precision    recall  f1-score   support

       False       0.68      0.79      0.73        80
        True       0.53      0.40      0.45        48

    accuracy                           0.64       128
   macro avg       0.61      0.59      0.59       128
weighted avg       0.63      0.64      0.63       128

